In [1]:
import os
from readlif.reader import LifFile
from tifffile import imwrite
import numpy as np

In [20]:
# full path to the lif file
file_path = r'r:\Narayanan\202409_Bcell\20240716_Bcell.lif'

# path to the folder where the tiff files will be saved
dir_save = r'r:\Narayanan\202409_Bcell\tifile'

In [3]:
# open lif image
image_stack = LifFile(file_path)

In [4]:
img_list = [[image.name, image.dims] for image in image_stack.get_iter_image()]
print(len(img_list))
img_list

254


[['5X DAPI/B/1', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/2', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/3', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/4', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/5', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/6', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/7', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/8', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/9', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/10', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/11', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/12', Dims(x=2048, y=2048, z=1, t=1, m=4)],
 ['5X DAPI/B/1_Merged', Dims(x=3808, y=3809, z=1, t=1, m=1)],
 ['5X DAPI/B/2_Merged', Dims(x=3808, y=3809, z=1, t=1, m=1)],
 ['5X DAPI/B/3_Merged', Dims(x=3809, y=3809, z=1, t=1, m=1)],
 ['5X DAPI/B/4_Merged', Dims(x=3809, y=3809, z=1, t=1, m=1)],
 ['5X DAPI/B/5_Merged', Dims(x=3809, y=3808, z=1, t=1, m=1)],
 ['5X DAPI/B/6_Merged', Dim

In [5]:
# find merged images that don't have matching tiles saved
img_list = [image for image in image_stack.get_iter_image() if (('Merged' in image.name) and not ('5X' in image.name) and not (image.name[:-7] in [image.name for image in image_stack.get_iter_image()]))]
print([(x.name,x.channels) for x in img_list])

[('R2_DAPI_CycD1-AF555_p21-AF647/1_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/2_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/3_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/4_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/5_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/6_Merged', 4), ('R2_DAPI_CycD1-AF555_p21-AF647/9_Merged', 4)]


In [6]:
# specify full names for the wells (missing from the images names)
well_list = ['D/1', 'D/2', 'D/3', 'D/4', 'D/5', 'D/6', 'D/9']

In [13]:
# calculate canvas size
TILE_SIZE = 2048
canvas_size = int(9*TILE_SIZE-8*TILE_SIZE*0.1)
canvas_size

16793

In [14]:
def cut_into_tiles_snaking(array, num_tiles=9, overlap=0.1):
    """Cut the array into tiles with snaking pattern"""
    
    tile_size = TILE_SIZE
    
    # Determine the stride size based on the overlap
    stride = int(tile_size * (1 - overlap))
    
    # Create a list to store the tiles
    tiles = []
    
    # Use nested loops to slide over the array and extract tiles
    for i in range(0, array.shape[0] - tile_size + 1, stride):
        row_tiles = []
        for j in range(0, array.shape[1] - tile_size + 1, stride):
            # Extract the tile from the array
            tile = array[i:i + tile_size, j:j + tile_size]
            row_tiles.append(tile)
        
        # Reverse the row for snaking effect on odd rows
        if (i // stride) % 2 == 1:
            row_tiles.reverse()
        
        tiles.extend(row_tiles)
    
    return tiles

In [22]:
for well,img in zip(well_list,img_list):
    print(f'Image name: {img.name}')    
    print(f'Well: {well}')
    print(f'Channels: {img.channels}')
    r = img.name.split('_')[0]
    print(f"Round: {r}")
    for ch in range(img.channels):
        
        # extract the image
        image = np.array(img.get_frame(z=0, t=0, c=ch, m=0))
        
        # position the image withing the canvas
        canvas = np.zeros((canvas_size,canvas_size),dtype=np.uint16)
        row_start = int(canvas_size//2 - image.shape[0]//2)
        col_start = int(canvas_size//2 - image.shape[1]//2)
        canvas[row_start:row_start+image.shape[0],col_start:col_start+image.shape[1]] = image

        tiles = cut_into_tiles_snaking(canvas)

        # save the tiles
        for ind,t in enumerate(tiles):
            imwrite(os.path.join(dir_save,f"{well.replace('/','')}_r_{str(r[1:]).zfill(2)}_ch_{str(ch).zfill(2)}_{str(ind).zfill(2)}.tif"), t.astype('uint16'))
        

Image name: R2_DAPI_CycD1-AF555_p21-AF647/1_Merged
Well: D/1
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/2_Merged
Well: D/2
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/3_Merged
Well: D/3
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/4_Merged
Well: D/4
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/5_Merged
Well: D/5
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/6_Merged
Well: D/6
Channels: 4
Round: R2
Image name: R2_DAPI_CycD1-AF555_p21-AF647/9_Merged
Well: D/9
Channels: 4
Round: R2


In [18]:
# len(tiles)
# for tile in tiles:
#     print(tile.shape)

81

### Compare with other stitched image

Test affinder as a solution

In [7]:
test_list = [image for image in image_stack.get_iter_image() if ('R2_DAPI_CycD1-AF555_p21-AF647/1_Merged' in image.name or 'R5_DAPI_CycA2-AF555_p16-AF647/D/1_Merged' in image.name)]
test_list

['LifImage object with dimensions: Dims(x=16218, y=16215, z=1, t=1, m=1)',
 'LifImage object with dimensions: Dims(x=16728, y=16727, z=1, t=1, m=1)']

In [9]:
import napari

viewer = napari.Viewer()
for im in test_list:
    viewer.add_image(np.array(im.get_frame(z=0, t=0, c=0, m=0)), name=im.name, blending='additive')